In [ ]:
# Imports
import pandas as pd
import numpy as np

In [ ]:
# Definitions
filename_full = r"../../data/data_full.xlsx"
filename = r"../../data/int_agreement (cleaned).xlsx"
sheet_names = ["Arto", "Evanfiya", "James", "Juho", "Sami"]


def load_spreadsheet(filepath,  add_index=False):
    sheets = pd.read_excel(filepath, sheet_name=sheet_names, index_col=0 if add_index else False)
    return sheets

def label_data(df, slicer, predicate, label="label", use_all=True):
    reducer = all if use_all else any
    labels = (df.iloc[slicer] # Pick relevant data
        .apply(predicate) # Apply predicate
        .apply(reducer, axis=1) # Reduce to single boolean column
        .apply(lambda label : "yes" if label else "no")) # Convert back to yes/no
    labels.name = label
    return labels

In [ ]:
sheets = load_spreadsheet(filename) # Load
df_agreement = pd.concat(sheets.values(), axis=0).dropna(axis=0, subset="title") # Combine sheets
#df_agreement

In [ ]:
slice_s = 11
slice_e = 14
eval_cols = df_agreement.columns[slice_s:slice_e]

# Use majority vote to decide label
aggregated = df_agreement[eval_cols].groupby(df_agreement.index).agg(lambda x: x.mode().iloc[0])

df_agreement = df_agreement.drop_duplicates("title") # Make sure only uniques
df_agreement[eval_cols] = aggregated[eval_cols] # Update columns

#df_agreement

In [ ]:
sheets = load_spreadsheet(filename_full)
df_full = pd.concat(sheets.values(), axis=0).dropna(axis=0, subset="title")

#Drop first column to match inter-rater cols
df_full = df_full[df_full.columns[1:]]

# df_full

In [ ]:
df_complete = pd.concat([df_full, df_agreement], ignore_index=True)
#df_complete.to_csv("complete_dataset.csv", sep=";", index=False)

In [ ]:
stats = df_full.columns[slice_s:slice_e]
columns = pd.MultiIndex.from_tuples([
    ("yes", "Count"),
    ("yes", "Percentage"),
    ("partially", "Count"),
    ("partially", "Percentage"),
    ("no", "Count"),
    ("no", "Percentage"),
])

len_df = len(df_complete)
count_yes = df_complete[stats].eq("yes").sum()
count_partially = df_complete[stats].eq("partially").sum()
count_no = df_complete[stats].eq("no").sum()

data = list(zip(
    count_yes, 100 * count_yes / len_df,
    count_partially, 100 * count_partially / len_df,
    count_no, 100 * count_no / len_df
))

df_stats = pd.DataFrame(data, index=s, columns=columns)
df_stats

#print(
#    df_stats.to_latex(float_format="%.1f")
#    .replace("tabular}", "tabularx}{\\textwidth}", 1)
#    .replace("tabular}", "tabularx}")
#)


#df[label].where(lambda x : x == "yes").dropna().count() / len(df[label])

In [ ]:
#result2.to_csv("out.csv", sep=";", columns=["title", "topic", "theme", "concept", "problemDescription", "exampleSolution", *eval_cols, label], index=False)
#result2.to_csv("int_agreement_majority_vote.csv", sep=";", columns=["title", "topic", "theme", "concept", "problemDescription", "exampleSolution", *eval_cols, label], index=False)